In [1]:
#################### SETTINGS ###########################

# import stuff and etc.
import pygame, sys
import time
import pygame.freetype
#pygame.mixer.pre_init(buffer=7000)
from pygame.locals import *
from pygame.math import Vector2 as vec
import copy

# these are the screen settings of pygame
TITLE = "In the Garden of Eden: A Maze Game"
WIDTH, HEIGHT = 651, 651
FPS = 60
TOP_BOTTOM_BUFFER = 62
MAZE_WIDTH, MAZE_HEIGHT = WIDTH-TOP_BOTTOM_BUFFER, HEIGHT-TOP_BOTTOM_BUFFER # (589, 589)
HS_FILE = "high score.txt"

# color settings
DARK_BLUE = (35, 37, 38)
ORANGE = (233, 116, 0)
RUST = (139, 47, 0)
GREEN = (94, 153, 24)
CREAM = (255, 250, 226)
APPLE_RED = (249, 75, 74)

# font settings
START_TEXT_SIZE = 30
START_TEXT_SIZE2 = 20
START_TEXT_SIZE3 = 10
START_TEXT_SIZE4 = 15
START_FONT = 'arial black'

#################### APP_CLASS ##########################

pygame.init()
# initialize pygame. If merong pygame functions na gilagay sa taas ng function nito, magerror ang code
# kasi need pa iinitialize ang pygame first

vec = pygame.math.Vector2 # can use for velocity, position, acceleration

class App:
    def __init__(self): # attributes
        self.music = pygame.mixer.Sound('grasstown maze bg music.mp3')
        self.music.set_volume(0.3)
        self.music.play()
        pygame.mixer.music.load('win.mp3')
        self.walk = pygame.mixer.Sound('walk.ogg')
        self.walk.set_volume(0.7)
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        self.clock = pygame.time.Clock()
        self.font=pygame.freetype.SysFont('arial black', 19)
        # difference between .freetype, .font, and .SysFont:
        # pygame.freetype directly accesses the freetype library and offers additional features such as
        # drawing text vertically
        # pygame.font uses the standard module for drawing text
        # SysFont is directly loading fonts from your oeprating system
        # font sa timer in-game
        
        self.font2=pygame.freetype.SysFont('arial black', 20)
        # font sa timer sa game over screen
        
        self.font.origin = True
        # origin() is a pygame function that if True, render_to() and render_raw_to() will take
        # the dest position to be that of the text origin, as opposed to the top-left corner of the
        # bounding box
        
        pygame.display.set_caption(TITLE)
        # title ng pygame window or ng game
        
        playerIcon = pygame.image.load('apple with leaf.png').convert_alpha()
        # convert_alpha() supports transparent images
        # convert() does not
        
        pygame.display.set_icon(playerIcon)
        # icon sa upper left ng pygame window
        
        self.running = True
        self.state = 'start'
        self.cell_width = MAZE_WIDTH//31 # 19
        self.cell_height = MAZE_HEIGHT//31 # 19
        self.walls = []
        self.points = []
        self.big_points = []
        self.time_points = []
        self.grass = []
        self.endpoints = []
        self.end_pos = []
        self.p_pos = None
        self.startCount = False
        # regardless if True or False, hindi parin magerror ang timer in-game kasi dahil sa time()
        # function
        # additionally, ang time() function nakalagay lang sa self.state = 'playing', which means na
        # hindi magstart count ng time itong function na ito galing sa title screen pa lang
        # giset lang natin na False just in case merong bug and nagstart pala siya count ng time
        # from the title screen mismo
        
        self.newTick = 0
        self.Tick = self.newTick
        self.seconds=0
        self.minutes=0  
        
        self.load()
        
        self.player = Player(self, vec(self.p_pos))
        # self.player will inherit the attributes of the Player class w/ a specific position (p_pos)
        
        self.make_endpoints()
              
        self.points_pic = pygame.image.load('apple1.png')
        self.big_points_pic = pygame.image.load('apple with leaf.png')
        self.grass_pic = pygame.image.load('grass.png')
        self.scaled_points = pygame.transform.scale(self.points_pic, (10,10))
        # (10,10) is the scaled size that we want
        # same concept for scaled images below
        
        self.scaled_big_points = pygame.transform.scale(self.big_points_pic, (17,17))
        self.scaled_grass = pygame.transform.scale(self.grass_pic, (20,20))
        
        self.time_points_pic = pygame.image.load('time.png')
        self.scaled_time_points = pygame.transform.scale(self.time_points_pic, (16,16))
        
        self.endpoint_pic = pygame.image.load('apple tree.png').convert_alpha()
        self.draw_endpoint = pygame.transform.scale(self.endpoint_pic, (65,65))
              
    def run(self): # behaviours
        while self.running:
            if self.state == 'start':
                self.start_events()
                self.start_draw()
                pygame.display.update()
                # updates parts of the screen all at once since it is more efficient than
                # having to update everytime there's a new object being called
                
            elif self.state == 'playing':
                pygame.mixer.music.play(0)
                # (0) is how many loops it will undergo - normal lang
                # if the loop is (-1), it's infinity loops
                
                self.playing_events()
                self.playing_update()
                self.playing_draw()
                self.time()
                pygame.display.update()
            elif self.state == 'game over':
                self.music.stop()
                # stops self.music which is grasstown mp3
                
                self.game_over_events()
                self.game_over_draw()
                pygame.display.update()
            else:
                self.running = False
            self.clock.tick(FPS)
            # while the game is running, it will operate at 60 FPS
        
        pygame.quit()

#################### HELPER FUNCTIONS ###################

    def draw_text(self, words, screen, pos, size, color, font_name,
                  centered=False):
        font = pygame.font.SysFont(font_name, size)
        text = font.render(words, True, color)
        # the middle part of render(), which is True/False is an antialias tool, which smooths out the
        # jagged corners of the text. Since it is False, we DON'T use the antialias tool
        
        text_size = text.get_size()
        # get_size() returns the (width, height) of the text
        
        if centered: # is True
            pos[0] = pos[0] - text_size[0]//2
            pos[1] = pos[1] - text_size[1]//2
            
        # pos[0] and pos[1] uses indexing but it's basically (pos[0], pos[1]) OR (x,y) coordinates of
        # the text. text_size[0]//2 gets the middle width of the text, text_size[0]//2 gets the middle
        # height.
        # pos[0] - text_size[0]//2 is like getting the center of the number 5. The result that we want
        # is 3, so we set pos[0] as 5 and text_size[0]//2 is 2. The result of this will now be our
        # new pos[0], which is the centered, x-value coordinate of the text.
        # The same is true for pos[1] = pos[1] - text_size[1]//2.
        
        screen.blit(text, pos)
        # the blit() function allows us to draw an image/text/etc. over another one
        # format: blit(source, dest, area=None, special_flags=0) -> Rect
        # the source is where we want to blit the image/text/etc.
        # dest is the coordinates of the upper left corner of the image/text/etc.
        # by default, objects are blitted at the position of the upper left corner of the object

    def load(self):
        self.background = pygame.image.load('maze1.png').convert_alpha()
        self.background = pygame.transform.scale(self.background, (MAZE_WIDTH, MAZE_HEIGHT))
        
        # Opening walls file and creating walls list w/ coordinates of walls
        with open("walls.txt", 'r') as file:
            # open() - opens the walls.txt file
            # 'r' is a function that "reads" our text file
            
            for yidx, line in enumerate(file):
            # enumerate() adss a counter to the results so if EXAMPLE:
            # lst_ = [hello, my, name, is, usha]
            # enumerate(lst_)
            # result: (0, hello), (1, my), (2, name), (3, is), (4, usha)
            # for yidx, line in enumerate(file): - the yidx is the counter starting from 0
            # line is kung ano ang line number niyas a walls.txt
            
                for xidx, char in enumerate(line):
                # for xidx, line in enumerate(file): xidx is the counter starting from 0
                # char is the character na nasa walls.txt,
                # either '1', 'P', 'A', or 'E' tas yung line is ano ang 
                
                    if char == "1":
                        self.walls.append(vec(xidx, yidx))
                        # appends the vector version of (x,y) coordinates of the walls sa self.walls
                        # list. vector objects are immutable or unable to be changed.
                        # the coordinates of our walls should forever remain unchanged so we used vec()
                        
                    elif char == "P":
                        self.points.append(vec(xidx, yidx))
                        self.grass.append(vec(xidx, yidx))
                    elif char == "B":
                        self.big_points.append(vec(xidx, yidx))
                        self.grass.append(vec(xidx, yidx))
                    elif char == "T":
                        self.time_points.append(vec(xidx, yidx))
                        self.grass.append(vec(xidx, yidx))
                    elif char == "A":
                        self.p_pos = [xidx, yidx]
                        # [5,13] ang self.p_pos instead na [6,14] kasi nagstart ang counting sa 0
                        
                        self.grass.append(vec(xidx, yidx))
                        # bakit [xidx, yidx] lang? or bakit list version ito at hindi vec?
                        # kasi for now, di pa natin need ng vector version ng coordinates
                        # magamit lang natin siya later, if gipasok na natin ang self.p_pos or (x,y)
                        # coordinates sa Player class
                        
                    elif char == "E":
                        self.end_pos.append(vec(xidx, yidx))
                        self.grass.append(vec(xidx, yidx))

        # Creating a high score database
        with open(HS_FILE, 'r+') as file:
        # opens HS_FILE, which is highscore.txt
        # 'r+' is a function that "reads and writes (or re-writes rather)" the contents of the file
        
            try:
            # (try:) lets you test a line of code
                self.highscore = int(file.read())
            # meaning nito, what if gidelete mo ang contents ng HS_FILE?
            # magerror yung code kasi wala siya ginaread na content and wala siya ginaconvert na
            # integer kasi again, wala content yung file mo
            # ang solution nito yung (except:) sa baba
            
            except:
            # (except:) is the same as exception. so if magerror ang code, dito siya magdiretso
                self.highscore = 0
            # gives you the option to set the self.highscore as 0 kung walang laman yung highscore.txt
            # so ngayon, di na magerror yung code
    
    def make_endpoints(self):
        for pos in self.end_pos:
        # self.end_pos contains the coordinates of the endpoint (yung apple tree) sa ating maze
        # technically, isa lang talaga ang coordinate natin sa self.end_pos
        # pero just in case lang, for every (x,y) coordinate na mahanap mo nasa self.end_pos list,
        # execute the code below
        
            self.endpoints.append(Endpoint(self, pos))
            # appends the attributes of the Endpoint class and its position to self.endpoints, which
            # is originally an empty list
            # so in the end, self.endpoints will inherit the characteristics of the Endpoint class

    def reset(self):
        self.seconds = 0
        self.minutes = 0
        self.newTick = 0
        # resets the in-game timer to 00:00
        
        self.player.lives = 1
        # when the player "dies", self.player.lives = 0
        # but after reset, magbalik siya to 1 to signify na "buhay" na character natin
        # if more than 1 ang self.player.lives, pag mapunta na sa apple tree yung player, hindi
        # agad maging game over ang screen.
        # for example, self.player.lives = 2. so if mapunta ka sa puno, ma-minus-an ng 1 ang
        # self.player.lives. So now, self.player.lives = 1. Since hindi pa man 0 ang life mo, hindi
        # ka pa game over. 
        
        self.player.current_score = 0
        # resets the current score to 0
        
        self.p_pos = None
        # for now, sets self.p_pos (or player position) to None
        # machange lang yan siya maya pag maopen na ang walls.txt file
        # if hindi ito None, at KUNYARI (2,26) gilagay mo diyan, ma-overwrite parin siya
        # dahil pag giopen mo na ang walls.txt file, ichange niya yung coordinate ng player using
        # the coordinates na mahanap mo sa wall.txt file
        # so in short, ang coordinate ng player sa wall.txt file siya heavily gadepend

        self.points = []
        # gawa muna ng empty list para diyan natin iappend ang coordinates ng points and grass later
        # when you go down the code, lahat daw ng coordinates na nasa loob ng list na ito,
        # magpakita siya ng grass and apple image

        with open("walls.txt", 'r') as file:
        # same concept as the [ with open(HS_FILE, 'r') as file: ] from earlier
        
            for yidx, line in enumerate(file):
                for xidx, char in enumerate(line):
                    if char == 'P':
                        self.grass.append(vec(xidx, yidx))
                        self.points.append(vec(xidx, yidx))
                    if char == "B":
                        self.big_points.append(vec(xidx, yidx))
                        self.grass.append(vec(xidx, yidx))
                    if char == "T":
                        self.time_points.append(vec(xidx, yidx))
                        self.grass.append(vec(xidx, yidx))
                    if char == 'A':
                        self.p_pos = [xidx, yidx] 
                        self.player = Player(self, vec(self.p_pos))
                    # you might be wondering, bakit 'P', 'A', 'B', and 'T' lang?
                    # Bakit hindi sali ang '1' and 'E'?
                    # kasi after playing the game, the only thing na nagchange lang is yung position
                    # ng player and yung points, big points, and time points na "gikain" mo
                    # never man nagchange ang location ng walls or ang location ng endpoint so di na
                    # natin siya kailangan i-reset
                    # bakit merong self.player = Player(self, vec(self.p_pos))?
                    # para mainherit ng self.player ang attributes and behaviours ng Player class
                    # and thus, meron siyang position, movement, etc.
                    
        self.state = 'playing'
        # after mareset yang lahat na nasa taas, magbalik to playing state ang game so makalaro ka ulit
        # if ang self.state = 'start', magbalik siya sa title screen ng game
        # if self.state = 'game over', forever mastuck ang screen mo sa game over screen at hindi ka
        # makalaro ulit

#################### INTRO FUNCTIONS ####################

    def start_events(self):
        for event in pygame.event.get():
        # pygame.event.get() - kunin niya lahat ng mga events na ginatry mo execute using your mouse,
        # keyboard, etc. so clicking on kunyari, a button, is an event. Pressing a letter sa keyboard
        # mo is also counted as an event.
        
            if event.type == pygame.QUIT:
            # if you click on the X button on the upper right corner ng pygame window, or the exit
            # button, i-execute niya ang code below
            
                self.running = False
                # when self.running is False, hindi na magrun ang game and magclose yung pygame window
                
            if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
            # pygame.KEYDOWN is the same as "pressing something". Combined with pygame.K_SPACE,
            # basically, it means na "if you press the space bar key on your keyboard"
            
                self.state = 'playing'
                # when self.state is playing, then pwede ka na makalaro kasi nandoon laht yung
                # functions, images, and events na nangyayari pag magplay ka
                
    def start_draw(self):
        self.screen.fill(DARK_BLUE)
        # fill() - fills the screen with a DARK_BLUE color
        # if kunyari, ichange mo yang DARK_BLUE tas gilagay mo RED, magerror siya. bakit?
        # kasi from the start, wala tayo nagset ng RGB color sa RED na variable
        # however, if sa color settings sa pinakataas ng code, nagset ka ng RED = (255, 0, 0)
        # then maging red ang color ng screen natin
        # self.screen is yung source ng kung asan natin ifill ng color
        
        self.draw_text('IN THE GARDEN OF EDEN', self.screen, [WIDTH//2,HEIGHT//2-75],
                       START_TEXT_SIZE, ORANGE, START_FONT, centered=True)
        # yung format ng draw_text() function natin is galing sa user-defined function na gigawa
        # natin sa taas
        # yang [WIDTH//2,HEIGHT//2-75] is ang (x,y) coordinate ng text natin
        # If [WIDTH//2,HEIGHT//2] lang siya, magshow up siya sa pinakacenter ng pygame screen natin
        # However, gusto ko man nasa center ang x coordinate niya tas medyo taas ng konti sa y
        # coordinate so nagminus ako ng 75 sa HEIGHT//2
        # the same applies for draw_text somethings below
        
        self.draw_text('A MAZE GAME', self.screen, [WIDTH//2,HEIGHT//2-50],
                       START_TEXT_SIZE2, CREAM, START_FONT, centered=True)
        self.draw_text('PRESS SPACE TO PLAY', self.screen, [WIDTH//2,HEIGHT//2+25],
                       START_TEXT_SIZE2, GREEN, START_FONT, centered=True)
        self.draw_text('USE ARROWS TO MOVE', self.screen, [WIDTH//2,HEIGHT//2+50],
                       START_TEXT_SIZE2, GREEN, START_FONT, centered=True)
        self.draw_text('PRESS Q TO EXIT', self.screen, [WIDTH//2,HEIGHT//2+75],
                       START_TEXT_SIZE2, GREEN, START_FONT, centered=True)
        self.draw_text('BY: COBRADO, MACALDO, ARRUBIO, & DEMORAL',
                       self.screen, [WIDTH//2,HEIGHT//2+125],
                       START_TEXT_SIZE3, CREAM, START_FONT, centered=True)
        self.draw_text('HIGH SCORE: {}'.format(self.highscore),
                       self.screen, [WIDTH//2, 20], START_TEXT_SIZE4,
                       GREEN, START_FONT, centered = True)
        # the HIGH SCORE contains values so naglagay ako ng {} to signify na sa loob niyan ilagay yung
        # values
        # format: 'string'.format(value1, value2...)
        # format() formats the specified value(s) and insert them inside the string's placeholder
        # the {} is the placeholder btw
        # tas ang values na iinsert mo daw diyan sa placeholder na yan is self.highscore
        # bakit [WIDTH//2, 20] lang at hindi something like [WIDTH//2, HEIGHT//2-something]?
        # kasi hindi ko alam ang definite na coordinate ng pinakataas ng screen so nagset nlng ako ng
        # number na sa tingin ko nasa taas na part

#################### PLAYING FUNCTIONS ##################

    def playing_events(self):
        for event in pygame.event.get():
        # same concept na giexplain ko sa start_events(self):
        
            if event.type == pygame.QUIT:
                self.running = False
                
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    self.player.move(vec(-1,0))
                    # vec(-1,0) allows you to move to the left 1 pixel at a time
                    self.walk.stop()
                    # stops the self.walk music
                    self.walk.play(-1)
                    # infinitely plays the self.walk music
                    
                elif event.key == pygame.K_RIGHT:
                    self.player.move(vec(1,0))
                    # vec(-1,0) allows you to move to the right 1 pixel at a time
                    self.walk.stop()
                    self.walk.play(-1)
                    
                elif event.key == pygame.K_UP:
                    self.player.move(vec(0,-1))
                    # vec(-1,0) allows you to move up 1 pixel at a time. -1 kasi baliktad ang concept
                    # niya sa programming na instead na positive dapat yan siya, negative na
                    self.walk.stop()
                    self.walk.play(-1)
                    
                elif event.key == pygame.K_DOWN:
                    self.player.move(vec(0,1))
                    # vec(-1,0) allows you to move down 1 pixel at a time. 1 kasi baliktad ang concept
                    # niya sa programming na instead na negative dapat yan siya, positive na
                    self.walk.stop()
                    self.walk.play(-1)
                    
                if event.key == pygame.K_q:
                    self.running = False
                # if you press the letter Q on your keyboard, self.running will be False, the game
                # will no longer run, and the pygame window will exit
                    
    def time(self):
        self.startCount = True
        if self.startCount: # is True
            self.newTick += 1 # increment 1 to the self.newTick counter
        second = int(self.newTick % 60)
        # modulus (%) divides and returns the remainder of the given numbers
        # so everytime maging 60 and self.newTick, ang remainder niya is 0
        # so gabalik/mag-reset siya to 0
        
        minute = int(self.newTick % 3600)
        # modulus (%) divides and returns the remainder of the given numbers
        # so everytime maging 3600 and self.newTick, ang remainder niya is 0
        # so gabalik/mag-reset siya to 0
        # bakit 3600? for every minute sa clock, there is 3600 seconds or ticks
        
        if second == 59:
            self.seconds += 1
        # bakit 59 lang?
        # kasi if second = 60, magreset ang second to 0 balik dahil sa % sa taas
        # so meaning niyan, everytime maglabas ang 0, magincrement ka ng 1 sa counter ng self.seconds
        # but hindi man yan pwede kasi remember, nagstart man ang time natin sa 0 originally
        # hindi man pwede na wala pa gani nagstart ang timer, meron ka na agad 1 second
        
        if self.seconds == 60:
            self.seconds = 0
        # resets the self.seconds timer to 0
        # follows the concept of time
        
        if minute == 3599:
            self.minutes += 1
        # bakit 3599 lang?
        # kasi if minute = 3599, magreset ang minute to 0 balik dahil sa % sa taas
        # so meaning niyan, everytime maglabas ang 0, magincrement ka ng 1 sa counter ng self.minutes
        # but hindi man yan pwede kasi remember, nagstart man ang time natin sa 0 originally
        # hindi man pwede na wala pa gani nagstart ang timer, meron ka na agad 1 minute
        
        out = '{minutes:02d}:{seconds:02d}'.format(minutes=self.minutes, seconds=self.seconds)
        # 2d signifies 2 digits, including and starting from yang 0 sa unahan ng 2d
        # ganito daw ang format ng string
        
        self.font.render_to(self.screen, (40, 23), out, pygame.Color(GREEN))
        # render_to() renders the text onto a surface
        # format: render_to(surf, dest, text, fgcolor=None, bgcolor=None, style=STYLE_DEFAULT,
        # rotation=0, size=0) -> Rect
        # yang (40, 23) is the dest or position on the screen
        # out is the text

    def playing_update(self):
        self.player.update()
        # self.player.update is yung (new) movement ng Player class
            
        for endpoint in self.endpoints:
        # again, isa lang man ang coordinate na nasa laman ng self.endpoints pero just in case,
        # naggamit lang tayo ng for loop
        
            if endpoint.grid_pos == self.player.grid_pos:
                self.remove_life()
            # if same daw ang position ng player at ng endpoint or if nareach na ng player yung
            # apple tree, remove a life (from the player) 

    def playing_draw(self):
        self.screen.fill(CREAM)
        self.screen.blit(self.background, (TOP_BOTTOM_BUFFER//2, TOP_BOTTOM_BUFFER//2))
        # (TOP_BOTTOM_BUFFER//2, TOP_BOTTOM_BUFFER//2) is the (x,y) coordinate ng CENTER ng
        # space between the edge of the maze and the size ng pygame screen
        # yung space gud sa top and bottom ng maze (di counted yung space sa sides)
        # divided by 2 kasi yan yung center
        
        self.draw_points()
        self.draw_text('CURRENT SCORE: {}'.format(self.player.current_score),
                       self.screen, [WIDTH//2, 16], 18,
                       GREEN, START_FONT, centered=True)

        self.player.draw()
        
        for endpoint in self.endpoints:
            endpoint.draw()
        # ito yung gisabi ko kanina, na for every coordinate na nasa self.endpoints na list,
        # idraw niya yung image ng endpoint (apple tree)

    def remove_life(self):
        self.player.lives -= 1
        if self.player.lives == 0:
            self.state = 'game over'
        # the player has 1 life sa game. so if you remove a life, diba 0 na. that means the player
        # is now "dead", which signifies the end of the game or "game over"

    def get_rect(self):
        return pygame.Surface.get_rect()
        # the get_rect() function is useful to get the rectangular area of an image
        # useful for getting the center (x,y) coordinate of an image
    
    def draw_points(self):
        
        # Draws the image of every grass tile on the maze
        for grass in self.grass:
            grass_rect = self.scaled_grass.get_rect()
            grass_rect.center = (int(grass.x*self.cell_width+self.cell_width//2+TOP_BOTTOM_BUFFER//2),
                int(grass.y*self.cell_height+self.cell_height//2+TOP_BOTTOM_BUFFER//2))
            self.screen.blit(self.scaled_grass, grass_rect)
        
        # REMEMBER:
        # grass.x = the x-coordinate of the position of the grass image
        # grass.y = the y-coordinate of the position of the grass image
        # self.cell_width = MAZE_WIDTH//31 (kasi our maze has 31x31 blocks) = 19
        # self.cell_width//2 = 9.5
        # self.cell_height = MAZE_HEIGHT//31 (kasi our maze has 31x31 blocks) = 19
        # self.cell_height//2 = 9.5
        # TOP_BOTTOM_BUFFER//2 = 31
        
        # Python follows PEMDAS rule so priority by order is: Parentheses, Exponentation,
        # Multiplication, Addition, and Subtraction
        
        # For short, int(grass.x*self.cell_width+self.cell_width//2+TOP_BOTTOM_BUFFER//2) is:
        # int[(grass.x*self.cell_width) + (self.cell_width//2) + (TOP_BOTTOM_BUFFER//2)]
        
        # (grass.x*self.cell_width) is kunin niya yung buong area na dapat icover ng grass image
        # (self.cell_width//2) is yung center ng block/cell na yun
        # (TOP_BOTTOM_BUFFER//2) is center ng top and bottom space between edge ng maze and end of
        # pygame screen. I think, pero HINDI ako sure, meron ito para iindicate na dapat within the
        # maze ang ating points ?
        
        # for example, coordinate ng 1st grass tile is (1, 1):
            # int[(grass.x*self.cell_width) + (self.cell_width//2) + (TOP_BOTTOM_BUFFER//2)]
            # int[(1*19) + (9.5) + (31)] = int[(19) + (9.5) + (31)] = int(59.5)
            # int() to signify na dapat whole number ang ating value
            # If you print(grass_rect.center), the 1st result is (59,59) which means Python rounds down?
            # so 59 (pixels?) ang final x-coordinate ng center ng grass_rect image natin
        
        # same process lang rin sa final y-coordinate niya
        # all in all, grass_rect returns: rect(x, y, width, height)
        # pero yang x and y subject to change siya kay probably ginaadjust siya ng code
        # in proportion to its width and height
        
        # for example, if print(grass_rect.center), one of the results is (59,59)
        # however, if you print(grass_rect), one of the results is rect(49, 49, 20, 20)
        # the most likely explanation is nagiba rin yung x and y kasi giscale mo ang image and mas
        # nagliit siya?
        
        # ang buong process nito same lang rin for the apple points, big points, and time point
        # image below
            
        # Draws the image of every point on the maze    
        for point in self.points:
            points_rect = self.scaled_points.get_rect()
            points_rect.center = (int(point.x*self.cell_width+self.cell_width//2+TOP_BOTTOM_BUFFER//2),
                int(point.y*self.cell_height+self.cell_height//2+TOP_BOTTOM_BUFFER//2))
            self.screen.blit(self.scaled_points, points_rect)
        
        for big_point in self.big_points:
            big_points_rect = self.scaled_big_points.get_rect()
            big_points_rect.center = (int(big_point.x*self.cell_width+self.cell_width//2+TOP_BOTTOM_BUFFER//2),
                int(big_point.y*self.cell_height+self.cell_height//2+TOP_BOTTOM_BUFFER//2))
            self.screen.blit(self.scaled_big_points, big_points_rect)
            
        for time_point in self.time_points:
            time_points_rect = self.scaled_time_points.get_rect()
            time_points_rect.center = (int(time_point.x*self.cell_width+self.cell_width//2+TOP_BOTTOM_BUFFER//2),
                int(time_point.y*self.cell_height+self.cell_height//2+TOP_BOTTOM_BUFFER//2))
            self.screen.blit(self.scaled_time_points, time_points_rect)

################## GAME OVER FUNCTIONS ##################

    def game_over_events(self):
        for event in pygame.event.get():
        # same concept sa giexplain ko sa start_events(self):
        
            if event.type == pygame.QUIT:
                self.running = False
                
            if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                self.reset()
                # resets everything under the reset() function, so it resets the time, player position,
                # points, at ang life ng player
                
                self.music.play()
                # plays the background music (grasstown) once you press space bar on your keyboard
                
            if event.type == pygame.KEYDOWN and event.key == pygame.K_q:
                self.running = False
                
    def game_over_draw(self):
        self.walk.stop()
        # stops the walking music if self.state = 'game over' since wala na man galakad player natin

        out = 'YOUR TIME IS {minutes:02d}:{seconds:02d}'.format(minutes=self.minutes, seconds=self.seconds)
        # out has the same concept as explain in time(self):
        
        self.screen.fill(CREAM)
        # fill() has the same concept as explained in start_draw(self):
        
        self.draw_text("GAME OVER", self.screen, [WIDTH//2, HEIGHT//2-75],
                        50, DARK_BLUE, START_FONT, centered=True)
        # draw_text() has the same concept as explained in start_draw(self):
        
        self.draw_text('CURRENT SCORE: {}'.format(self.player.current_score), self.screen,
                        [WIDTH//2, HEIGHT//2-25], 20, GREEN, START_FONT, centered=True)
        self.draw_text("PRESS SPACE TO PLAY AGAIN", self.screen, [WIDTH//2, HEIGHT//2+75], 15,
                        RUST, START_FONT, centered=True)
        self.draw_text("PRESS Q TO EXIT", self.screen, [WIDTH//2, HEIGHT//2+90], 15,
                        RUST, START_FONT, centered=True)
        self.font2.render_to(self.screen, (215, HEIGHT//2-10), out, pygame.Color(GREEN))
        
        #Checking if the player's current score exceeds the high score
        if self.player.current_score > self.highscore:
            self.highscore = self.player.current_score
            # if your current score is greater than the high score recorded last time,
            # yung bago mong high score is yung current score na yun

            with open(HS_FILE, 'r+') as file:
                file.write(str(self.player.current_score))
            # opens the highscore.txt again and this time, it reads the high score recorded there
            # sa file and if mas mataas yung current score mo ngayon, i-re-write niya yung score sa
            # highscore.txt and yung current score mo ngayon ang isulat niya doon
            # str version ng current score and isulat niya kasi text file man ang gamit natin
            
        self.draw_text('HIGH SCORE: {}'.format(self.highscore), self.screen,
                            [WIDTH//2, HEIGHT//2+25], 20, ORANGE, START_FONT, centered=True)
        # 'string'.format() has the same concept as explain is start_draw(self):


pygame 2.1.0 (SDL 2.0.16, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import pygame
vec = pygame.math.Vector2

class Player:
    def __init__(self, app, pos):
        self.frame = 0
        self.eatapple = pygame.mixer.Sound('orb.ogg')
        self.eatapple.set_volume(0.3)
        self.eatbigapple = pygame.mixer.Sound('big_point.mp3')
        self.eatbigapple.set_volume(0.4)
        self.rewind = pygame.mixer.Sound('rewind.mp3')
        self.rewind.set_volume(0.3)
        self.app = app
        self.grid_pos = vec(pos[0], pos[1])
        # result: 5.0 13.0 or (5,13) - original grid position ng player
        # galing ang result sa self.p_pos ang self.player = Player(self, vec(p_pos))
        # (pos[0], pos[1]) uses indexing but it's basically the (x,y) coorindate ng grid/block kung
        # asan yung player
        
        self.pix_pos = self.get_pix_pos()
        # print(self.pix_pos) = [135, 287] - original pixel positionn ng player
        
        self.direction = vec(1,0)
        # that is which direction siya magstart initially move
        # vec(1,0) is magstart siya move sa right but since may wall man next to it, mukhang wala siya
        # gamove
        
        # if vec(0,0) siya, hindi talaga siya makamove
        # if vec(-1,0) siya, magstart siya move sa left pero di niya madetect instantly ang wall ng
        # maze and baka yung player mapunta sa labas ng maze
        # if vec(0,1), magstart siya move down even though wala pa gipress ng player ang keys
        # if vec(0,-1), magstart siya move up pero same mangyari sa vec(-1,0), di niya madetect agad
        # yung wall so magpass through siya sa walls
        # if vec(1,2) or any vec(x,y), magmove siya diagonally instead na left, right, up, down lang
        # if vec(2,0), vec(0,2) or any other number na di (1,0), di makamove ng tarong ang ating player
        
        # bale, if vec(x,y) where x and y is < 1 and > -1, magbilis ang movement ng player, pag
        # i-press mo ang keys, may chance na di maggana, and magpass through player natin sa walls,
        # may chance na diagonally siya magmove instead na left, right, up, and down lang, and di mo
        # makain yung points
        
        # if vec(x,y) where x and y is 0 naman, player can't move at all no matter how you press the
        # keys
        
        self.stored_direction = None
        self.able_to_move = True
        # regardless if True or False ito, makagalaw ka pa rin for some reason
        # kung vec(x,y) ito, makamove ka parin ng tarong
        # mainly because of the line self.able_to_move = self.can_move() sa update(self): below
        # ng Player class
        # nice siya kay di masira ang code XD
        
        self.current_score = 0
        self.speed = 1.5
        self.lives = 1
        self.player_movement = 1
        # if self.player_movement = 1, that means ang ANIMATION niya is yung default standing lang
    
    def update(self):
        if self.able_to_move: # is True
            self.pix_pos += self.direction*self.speed
            # self.pix_pos = self.pix_pos + (self.direction*self.speed)
            # (self.direction*self.speed) para magbilis ang player movement
            # kasi if self.speed = 1 lang, mabagal mashado ang player
            # self.pix_pos = self.pix_pos + (self.direction*self.speed) can be described as:
            # "the speed at which the player is moving at a certain direction per pixel"
            
            # this is the part of the code where gaupdate ang each pixel position ng player
            # everytime gamove siya so gachange ang (x,y) pixel coordinates ng player if it moves
            # since magchange na ngayon ang pixel position coordinates natin, magchange rin ang ating
            # grid position coordinates
            
        if self.time_to_move(): # is True ?
            if self.stored_direction != None:
            # self.stored_direction will not equal to None once you move because of
            # the move(self, direction): function
            # sa move(self, direction): gistore ang current vec(x,y) coordinate ng player tas
            # yung value na yun ang gigamit rin for self.stored_direction
            
                self.direction = self.stored_direction
                # the self.direction (or current direction) now has the value ng (x,y) coordinate ng
                # self.stored_direction
                
            self.able_to_move = self.can_move()
            # we are able_to_move if can_move is True. Otherwise, if can_move is False, we are not
            # able to move
        
        ### Setting grid position in reference to pix pos
        self.grid_pos[0] = (self.pix_pos[0]-TOP_BOTTOM_BUFFER +
                            self.app.cell_width//2)//self.app.cell_width+1

        # self.grid_pos[0] basically returns the x-coordinate ng grid/block ng player
        
        # REMEMBER:
        # self.app.cell_width = MAZE_WIDTH//31 (kasi our maze has 31x31 blocks) = 19
        # self.app.cell_width//2 = 9.5
        # self.app.cell_height = MAZE_HEIGHT//31 (kasi our maze has 31x31 blocks) = 19
        # self.app.cell_height//2 = 9.5
        # TOP_BOTTOM_BUFFER = 62
        # TOP_BOTTOM_BUFFER//2 = 31
        
        # the equation is basically:
        # self.grid_pos[0] = (((self.pix_pos[0]-TOP_BOTTOM_BUFFER) + self.app.cell_width//2)//self.app.cell_width)+1
        # self.grid_pos[0] = (((self.pix_pos[0]-62) + 9.5)//19)+1
        
        # for example: self.pix_pos = [135,287]:
            # self.grid_pos[0] = (((135-62) + 9.5)//19)+1
            # self.grid_pos[0] = 5.34
            # tapos round down I think kasi if print(self.grid_pos[0]), result is 5.0
        
        # this whole process is the same for self.grid_pos[1] below
        
        self.grid_pos[1] = (self.pix_pos[1]-TOP_BOTTOM_BUFFER +
                            self.app.cell_height//2)//self.app.cell_height+1
        
        # print(self.grid_pos[0], self.grid_pos[1]) - 5.0 13.0 or (5,13) ang grid position ng
        # starting position ng player
        
        # if you combine the process ng self.grid_pos[0], self.grid_pos[1] dito, ang overall result
        # nito is the (x,y) grid coordinate kung asan ang player currently
        
        # so if magchange ang pixel position ng player, magchange rin itong grid position
        
        if self.on_point(): # is True
            self.eat_point()
        if self.on_big_point(): # is True
            self.eat_big_point()
        if self.on_time_point(): # is True
            self.eat_time_point()
        
    def draw(self):
        keys = pygame.key.get_pressed()
        # pygame.key.get_pressed() - gets the state of all keyboard buttons (if what keys are being
        # pushed/pressed or not)
        
        moveForward = [pygame.image.load('forwardLeft.png'), pygame.image.load('forwardRight.png')] 
        moveBackward = [pygame.image.load('backwardLeft.png'), pygame.image.load('backwardRight.png')]
        moveLeft = [pygame.image.load('leftLeft.png'), pygame.image.load('leftRight.png')]
        moveRight = [pygame.image.load('rightLeft.png'), pygame.image.load('rightRight.png')]
        stand = pygame.image.load('stance.png').convert_alpha()
        player_pic = stand
            
        if self.frame + 1 >= 30:
        # bakit 30?
        # kasi per frame lasts 15 ticks and there are only 2 images per animation
        # 15 x 2 = 30
        # bakit self.frame + 1 >= 30: ?
        # just in case meron bug and naging more than 30 ang frames
        
                self.frame = 0
                # resets the number of frames to 0
        
        if keys[pygame.K_UP]: # if the key being pressed is the up arrow, execute the code below
            self.player_movement = 2
            player_pic = (moveForward[self.frame//15])
            # self.frame will continue to remain at index[0] or the 1st character animation image
            # unless magaccumulate ang counter ng self.frame and maging 15 na siya, which will then
            # allow the character's animation to proceed to the next character animation image, which
            # is index[1]
            # para magbalik to the 1st character animation image ang frame, the magic happens
            # sa code sa taas, yung if self.frame + 1 >= 30:
            # yan ang gareset ng self.frame balik to 0 so magbalik rin ang 1st character animation
            # image natin
            
            self.frame += 1
            # counter para macheck natin if ilang ticks na ba ang nagpass
            
            # this whole process repeat for the same line of code sa baba
            
        elif keys[pygame.K_DOWN]:
            self.player_movement = 3
            player_pic = (moveBackward[self.frame//15])
            self.frame += 1
        elif keys[pygame.K_LEFT]:
            self.player_movement = 4
            player_pic = (moveLeft[self.frame//15])
            self.frame += 1
        elif keys[pygame.K_RIGHT]:
            self.player_movement = 5
            player_pic = (moveRight[self.frame//15])
            self.frame +=1
        else:
        # if wala gipress ang keyboard buttons, the code below allows our character's animation to
        # continue looping
            
            # bale, same process lang rin sa taas
            if self.able_to_move: # is True
                if self.player_movement == 2: # which is player movement if you pressed the UP key
                    player_pic = (moveForward[self.frame//15])
                    self.frame += 1
                elif self.player_movement == 3: # which is player movement if you pressed the DOWN key
                    player_pic = (moveBackward[self.frame//15])
                    self.frame += 1
                elif self.player_movement == 4: # which is player movement if you pressed the LEFT key
                    player_pic = (moveLeft[self.frame//15])
                    self.frame += 1
                elif self.player_movement == 5: # which is player movement if you pressed the RIGHT key
                    player_pic = (moveRight[self.frame//15])
                    self.frame +=1
            else: # if self.able_to_move is False or if nakabangga yung player ng wall
                self.player_movement = 1 # which is the simple, standing animation lang
                
        draw_player = pygame.transform.scale(player_pic, (30,30))
        # scales the image with the size of 30x30 pixels
        
        player_rect = draw_player.get_rect()
        # gets the rectangular area of the image
        
        player_rect.center = int(self.pix_pos.x), int(self.pix_pos.y)-8
        # gets the center of the rectangular area of the image
        # meron diyang minus 8 kasi gusto ko yung paa ng player ang maging center
        
        self.app.screen.blit(draw_player, player_rect)
        # blits the player image with a specific rectangular area (and center ng rectangle na yun)
        
        # Drawing player lives
        for x in range(self.lives):
            pygame.draw.circle(self.app.screen, CREAM, (37, HEIGHT-16), 7)
            # cream lang ang color para hindi mo siya makita sa game but it's there
        
    def on_point(self):
        if self.grid_pos in self.app.points:
        # if ang grid position ng player meron sa self.app.points list, execute the code below
            
            if int(self.pix_pos.x) % self.app.cell_width//2-1 == 0:
                if self.direction == vec(1,0) or self.direction == vec(-1,0):
                # if the player's direction is RIGHT or LEFT, execute the code below
                
                    return True
                    # return True - meaning, same ang coordinates ng player and ng points na nasa
                    # self.app.points list
            
            # int(self.pix_pos.x) - the x-coordinate ng pixel position ng player
            # self.app.cell_width//2-1 - center ng cell width tapos minus 1 kasi kunin niya yung edge
            # ng cell
            # bakit need niya kunin yung edge?
            # kasi doon lang natin maassure na completely covered ng player yung position ng point at
            # hindi lang yung edge ng point ang natouch ng player
            
            # example: self.pix_pos = [135,287]:
            # self.app.cell_width//2 = 9.5-1 = 8.5
            # is 135 % 8.5 == 0? No. So wala yung player position niya sa point.
            
            # as for the reason bakit modulus, HINDI ako exactly sure kung bakit yan ang gigamit
            # and hindi ko rin exactly mavisualize ang code BUT I think it's to check if fully na ba
            # nakain ng player ang apple
            
            # repeat the same process for the code below
            
            if int(self.pix_pos.y) % self.app.cell_height//2-1 == 0:
                if self.direction == vec(0,1) or self.direction == vec(0,-1):
                # if the player's direction is DOWN or UP, execute the code below
                
                    return True
        return False
        # otherwise, if the player's pixel position isn't the same as the position ng points,
        # False ang ireturn niya, which means wala ka nasa coordinate ng point na yun
    
    # repeat the same process ng on_point(self): for the codes below
    
    def on_big_point(self):
        if self.grid_pos in self.app.big_points:
            if int(self.pix_pos.x) % self.app.cell_width//2-1 == 0:
                if self.direction == vec(1,0) or self.direction == vec(-1,0):
                    return True
            if int(self.pix_pos.y) % self.app.cell_height//2-1 == 0:
                if self.direction == vec(0,1) or self.direction == vec(0,-1):
                    return True
        return False

    def on_time_point(self):
        if self.grid_pos in self.app.time_points:
            if int(self.pix_pos.x) % self.app.cell_width//2-1 == 0:
                if self.direction == vec(1,0) or self.direction == vec(-1,0):
                    return True
            if int(self.pix_pos.y) % self.app.cell_height//2-1 == 0:
                if self.direction == vec(0,1) or self.direction == vec(0,-1):
                    return True
        return False
    
    def eat_point(self):
        self.eatapple.play()
        # everytime the eat_point(self): function is called, magplay yung eatapple sound (orb.ogg)
        
        self.app.points.remove(self.grid_pos)
        # removes the (x,y) grid coordinate ng specific point na yun sa list of points, which is nasa
        # self.app.points
        # remove() function removes an object from the list, useful when you know the object but
        # not its index
        
        self.current_score += 1
        # everytime a ppoint is "eaten", increments 1 to the current score counter
    
    # repeat the same process ng eat_point(self): for the codes below
    
    def eat_big_point(self):
        self.eatbigapple.play() # music
        self.app.big_points.remove(self.grid_pos)
        self.current_score += 5
        # adds 5 points to the score of the player instead na 1. game feature
        
    def eat_time_point(self):
        self.rewind.play() # music
        self.app.time_points.remove(self.grid_pos)
        self.app.seconds -= 5
        # minuses 5 secs to the time of the player. game feature
    
    def move(self, direction):
        self.stored_direction = direction
        # itong move(self, direction) function is ginagamit sa playing_events(self):
        # so yung vec(x,y) na makuha mo doon sa playing_events(self), yung yung maging value/new
        # coordinate ng self.stored_direction mo
        # bale, sa playing_events(self) naprovide yung value
        # tas yung value na yun ang gi-store sa self.stored_direction function
    
    def get_pix_pos(self):
        return vec((self.grid_pos[0]*self.app.cell_width)+TOP_BOTTOM_BUFFER//2+self.app.cell_width//2,
                   (self.grid_pos[1]*self.app.cell_height) + TOP_BOTTOM_BUFFER//2+self.app.cell_height//2)
        # self.grid_pos[0] and self.grid_pos[1] uses indexing but it's the same as (x,y) coordinates
        # ng grid mismo
        # overall, it returns the (x,y) measurement ng entire area ng grid in pixel form
        
        # REMEMBER:
        # self.app.cell_width = MAZE_WIDTH//31 (kasi our maze has 31x31 blocks) = 19
        # self.app.cell_width//2 = 9.5
        # self.app.cell_height = MAZE_HEIGHT//31 (kasi our maze has 31x31 blocks) = 19
        # self.app.cell_height//2 = 9.5
        # TOP_BOTTOM_BUFFER//2 = 31
        
        # (self.grid_pos[0]*self.app.cell_width) - kunin niya yung buong width ng grid na dapt covered
        # by the pixels
        # (TOP_BOTTOM_BUFFER//2) - center ng top and bottom space between edge ng maze and end of
        # pygame screen
        # (self.app.cell_width//2) - center ng grid/cell/block na yun
        
        # the same thing applies for the y-coordinate, which is yung sa self.grid_pos[1] na part
        
        # example: grid_pos = [5,13]:
            # (self.grid_pos[0]*self.app.cell_width)+(TOP_BOTTOM_BUFFER//2)+(self.app.cell_width//2)
            # (5*19)+(31)+(9.5) = 135.5
            # (self.grid_pos[1]*self.app.cell_height)+(TOP_BOTTOM_BUFFER//2)+(self.app.cell_height//2)
            # (13*19)+(31)+(9.5) = 287.5
            
            # tapos i-round down both, I think kasi if i-print mo ang get_pix_pos(), ang maging result is:
            # [135,287] <--- ito rin ang self.pix_pos ng Player natin
        
        # REMINDER:
        # result nito changes everytime the grid position is changing also
        
    def time_to_move(self):
        
        # the process for this is the same as on_point(self):
        if int(self.pix_pos.x) % self.app.cell_width//2-1 == 0:
            if self.direction == vec(1,0) or self.direction == vec(-1,0):
                return True
        
        if int(self.pix_pos.y) % self.app.cell_height//2-1 == 0:
            if self.direction == vec(0,1) or self.direction == vec(0,-1):
                return True
    
    def can_move(self):
        # Detects whether the direction of the player is going through a wall
        for wall in self.app.walls:
        # for every (x,y) coordinate ng walls sa self.app.walls list, execute the code below
        
            if vec(self.grid_pos+self.direction) == wall:
            # if the player's grid position and direction is the (x,y) coordinate sa taas, execute
            # the code below
            
                self.app.walk.stop()
                # stops the walking music
                
                return False
                # return False - meaning, our player can't move. It can only move if it returns True
        
        return True
        # otherwise, if ang (x,y) grid coordinate and position ng player is NOT equal to a wall,
        # it returns True
        
        # ito ang main reason bakit makagalaw parin ng tarong ang ating player if ang
        # self.able_to_move mo is False or vec(x,y) sa __init__(self, app, pos): ng Player class
        # kasi no matter what, True parin ang ireturn niya dito sa can_move(self):
        
        # if False ito, doon na di talaga makamove ang character natin
        # if vec(x,y) ito, makamove parin ng tarong ang player natin for some reason?

In [3]:
import pygame
vec = pygame.math.Vector2

class Endpoint:
    def __init__(self, app, pos):
        self.app = app
        self.grid_pos = pos
        self.pix_pos = self.get_pix_pos()
    
    def get_pix_pos(self):
        
        # has the same concept and proceess sa get_pix_pos(self) ng Player class
        return vec((self.grid_pos[0]*self.app.cell_width)+TOP_BOTTOM_BUFFER//2+self.app.cell_width//2,
                    (self.grid_pos[1]*self.app.cell_height) + TOP_BOTTOM_BUFFER//2+self.app.cell_height//2)
        
    def draw(self):
        
        # has the same concept and process sa draw(self): ng Player class, yung sa baba banda
        endpoint_rect = self.app.draw_endpoint.get_rect()
        endpoint_rect.center = int(self.pix_pos.x), int(self.pix_pos.y)-25
        self.app.screen.blit(self.app.draw_endpoint, endpoint_rect)

In [4]:
#################### MAIN ####################
app = App() # assigns the App class to a variable
app.run() # runs the game